In [1]:
import pandas as pd

In [61]:
import numpy as np
from  multiprocessing import Pool

In [3]:
import geopy.distance as geodis

In [133]:
path = "E:/tableau_data/"
local_path = './data/'
bike_file = "2019-citi-data.csv"
bike_file_with_date = "2019-citi-data_with_date.csv"
weather_file = "NYC_weather_data_2019.csv"
weather_bike_file = '2019_citi_data_weather.csv'
bike_station_id_2019 = 'bike_station_id_name_index_2019.csv'
bike_station_id_2020 = 'bike_station_id_name_index_2020.csv'
trip_count_average_duration_distance = 'average_trip_duration_count_distance_2019.csv'
unique_trips_distance = 'unique_trips_distance_2019.csv'
daily_trips_duration_count = 'daily_trips_duration_count_2019.csv'
gender_count = 'date_gender_count.csv'
usertype_count = 'date_usertype_count.csv'

In [5]:
bike_df = pd.read_csv(path+bike_file)

In [6]:
weather_df = pd.read_csv(path + weather_file)

In [7]:
bike_df

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551692,2019-12-31 23:58:18.0160,2019-12-31 23:59:51.6360,3141.0,1 Ave & E 68 St,40.765005,-73.958185,3142.0,1 Ave & E 62 St,40.761227,-73.960940,28734,Subscriber,1985,2
20551693,2019-12-31 23:58:59.8960,2020-01-01 00:12:06.3460,490.0,8 Ave & W 33 St,40.751551,-73.993934,513.0,W 56 St & 10 Ave,40.768254,-73.988639,41418,Subscriber,1991,1
20551694,2019-12-31 23:59:03.6950,2020-01-01 00:04:54.8730,3349.0,Grand Army Plaza & Plaza St West,40.672968,-73.970880,3368.0,5 Ave & 3 St,40.672816,-73.983524,27746,Subscriber,1935,2
20551695,2019-12-31 23:59:21.3610,2020-01-01 00:25:32.9420,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,28506,Subscriber,1980,1


In [8]:
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])

In [9]:
bike_df['date'] = bike_df['starttime'].dt.date

In [10]:
bike_df['duration(hr)'] = (bike_df['stoptime'] - bike_df['starttime']).dt.seconds / 3600

In [11]:
bike_df['date']


0           2019-01-01
1           2019-01-01
2           2019-01-01
3           2019-01-01
4           2019-01-01
               ...    
20551692    2019-12-31
20551693    2019-12-31
20551694    2019-12-31
20551695    2019-12-31
20551696    2019-12-31
Name: date, Length: 20551697, dtype: object

In [12]:
bike_df.head()

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,duration(hr)
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160.0,Central Park West & W 76 St,40.778968,-73.973747,3283.0,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,2019-01-01,0.088889
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519.0,Pershing Square North,40.751873,-73.977706,518.0,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,2019-01-01,0.087778
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171.0,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154.0,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,2019-01-01,0.164167
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504.0,1 Ave & E 16 St,40.732219,-73.981656,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,2019-01-01,0.755278
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229.0,Great Jones St,40.727434,-73.993790,503.0,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,2019-01-01,0.084167


In [13]:
bike_df['start station id'].dtypes

dtype('float64')

In [14]:
#there are some station ids that are not numbers.  Need to fix.
bike_df['start station id'].fillna(0, inplace = True)
bike_df['end station id'].fillna(0, inplace = True)

In [15]:
bike_df['end station id'].isna()

0           False
1           False
2           False
3           False
4           False
            ...  
20551692    False
20551693    False
20551694    False
20551695    False
20551696    False
Name: end station id, Length: 20551697, dtype: bool

In [16]:
#check that there are no more na
k = 0
for row in bike_df['end station id']:
    k=k+1
    if pd.isna(row):
        print(k,row)

In [17]:
#convert to int
bike_df['start station id'] = bike_df['start station id'].astype(int)

In [18]:
#convert to int
bike_df['end station id'] = bike_df['end station id'].astype(int)

In [19]:
bike_df

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,duration(hr)
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160,Central Park West & W 76 St,40.778968,-73.973747,3283,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,2019-01-01,0.088889
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519,Pershing Square North,40.751873,-73.977706,518,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,2019-01-01,0.087778
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,2019-01-01,0.164167
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504,1 Ave & E 16 St,40.732219,-73.981656,3709,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,2019-01-01,0.755278
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229,Great Jones St,40.727434,-73.993790,503,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,2019-01-01,0.084167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551692,2019-12-31 23:58:18.016,2019-12-31 23:59:51.636,3141,1 Ave & E 68 St,40.765005,-73.958185,3142,1 Ave & E 62 St,40.761227,-73.960940,28734,Subscriber,1985,2,2019-12-31,0.025833
20551693,2019-12-31 23:58:59.896,2020-01-01 00:12:06.346,490,8 Ave & W 33 St,40.751551,-73.993934,513,W 56 St & 10 Ave,40.768254,-73.988639,41418,Subscriber,1991,1,2019-12-31,0.218333
20551694,2019-12-31 23:59:03.695,2020-01-01 00:04:54.873,3349,Grand Army Plaza & Plaza St West,40.672968,-73.970880,3368,5 Ave & 3 St,40.672816,-73.983524,27746,Subscriber,1935,2,2019-12-31,0.097500
20551695,2019-12-31 23:59:21.361,2020-01-01 00:25:32.942,252,MacDougal St & Washington Sq,40.732264,-73.998522,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,28506,Subscriber,1980,1,2019-12-31,0.436389


In [20]:
bike_df['usertype'].unique()

array(['Subscriber', 'Customer'], dtype=object)

In [21]:
bike_df

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,duration(hr)
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160,Central Park West & W 76 St,40.778968,-73.973747,3283,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,2019-01-01,0.088889
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519,Pershing Square North,40.751873,-73.977706,518,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,2019-01-01,0.087778
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,2019-01-01,0.164167
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504,1 Ave & E 16 St,40.732219,-73.981656,3709,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,2019-01-01,0.755278
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229,Great Jones St,40.727434,-73.993790,503,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,2019-01-01,0.084167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551692,2019-12-31 23:58:18.016,2019-12-31 23:59:51.636,3141,1 Ave & E 68 St,40.765005,-73.958185,3142,1 Ave & E 62 St,40.761227,-73.960940,28734,Subscriber,1985,2,2019-12-31,0.025833
20551693,2019-12-31 23:58:59.896,2020-01-01 00:12:06.346,490,8 Ave & W 33 St,40.751551,-73.993934,513,W 56 St & 10 Ave,40.768254,-73.988639,41418,Subscriber,1991,1,2019-12-31,0.218333
20551694,2019-12-31 23:59:03.695,2020-01-01 00:04:54.873,3349,Grand Army Plaza & Plaza St West,40.672968,-73.970880,3368,5 Ave & 3 St,40.672816,-73.983524,27746,Subscriber,1935,2,2019-12-31,0.097500
20551695,2019-12-31 23:59:21.361,2020-01-01 00:25:32.942,252,MacDougal St & Washington Sq,40.732264,-73.998522,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,28506,Subscriber,1980,1,2019-12-31,0.436389


In [22]:
bike_df['subscriber'] = 0
bike_df['customer'] = 0
bike_df['male'] = 0
bike_df['female'] = 0

In [23]:
bike_df

,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date,duration(hr),subscriber,customer,male,female
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,3160,Central Park West & W 76 St,40.778968,-73.973747,3283,W 89 St & Columbus Ave,40.788221,-73.970416,15839,Subscriber,1971,1,2019-01-01,0.088889,0,0,0,0
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,519,Pershing Square North,40.751873,-73.977706,518,E 39 St & 2 Ave,40.747804,-73.973442,32723,Subscriber,1964,1,2019-01-01,0.087778,0,0,0,0
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,3171,Amsterdam Ave & W 82 St,40.785247,-73.976673,3154,E 77 St & 3 Ave,40.773142,-73.958562,27451,Subscriber,1987,1,2019-01-01,0.164167,0,0,0,0
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,504,1 Ave & E 16 St,40.732219,-73.981656,3709,W 15 St & 6 Ave,40.738046,-73.996430,21579,Subscriber,1990,1,2019-01-01,0.755278,0,0,0,0
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,229,Great Jones St,40.727434,-73.993790,503,E 20 St & Park Ave,40.738274,-73.987520,35379,Subscriber,1979,1,2019-01-01,0.084167,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20551692,2019-12-31 23:58:18.016,2019-12-31 23:59:51.636,3141,1 Ave & E 68 St,40.765005,-73.958185,3142,1 Ave & E 62 St,40.761227,-73.960940,28734,Subscriber,1985,2,2019-12-31,0.025833,0,0,0,0
20551693,2019-12-31 23:58:59.896,2020-01-01 00:12:06.346,490,8 Ave & W 33 St,40.751551,-73.993934,513,W 56 St & 10 Ave,40.768254,-73.988639,41418,Subscriber,1991,1,2019-12-31,0.218333,0,0,0,0
20551694,2019-12-31 23:59:03.695,2020-01-01 00:04:54.873,3349,Grand Army Plaza & Plaza St West,40.672968,-73.970880,3368,5 Ave & 3 St,40.672816,-73.983524,27746,Subscriber,1935,2,2019-12-31,0.097500,0,0,0,0
20551695,2019-12-31 23:59:21.361,2020-01-01 00:25:32.942,252,MacDougal St & Washington Sq,40.732264,-73.998522,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,28506,Subscriber,1980,1,2019-12-31,0.436389,0,0,0,0


In [115]:
bike_df.columns


Index(['starttime', 'stoptime', 'start station id', 'start station name',
       'start station latitude', 'start station longitude', 'end station id',
       'end station name', 'end station latitude', 'end station longitude',
       'bikeid', 'usertype', 'birth year', 'gender', 'date', 'duration(hr)',
       'subscriber', 'customer', 'male', 'female'],
      dtype='object')

In [140]:
gender_subscriber_df = bike_df[['date','gender','usertype', 'start station id']]

In [141]:
gender_subscriber_df

,date,gender,usertype,start station id
0,2019-01-01,1,Subscriber,3160
1,2019-01-01,1,Subscriber,519
2,2019-01-01,1,Subscriber,3171
3,2019-01-01,1,Subscriber,504
4,2019-01-01,1,Subscriber,229
...,...,...,...,...
20551692,2019-12-31,2,Subscriber,3141
20551693,2019-12-31,1,Subscriber,490
20551694,2019-12-31,2,Subscriber,3349
20551695,2019-12-31,1,Subscriber,252


In [177]:
gender_count_df = gender_subscriber_df.groupby(['date','gender', 'start station id']).count().rename(columns = {'usertype': 'gCount'}).reset_index(level = [1, 2])

In [178]:
gender_count_df

,gender,start station id,gCount
date,,,
2019-01-01,0,72,1
2019-01-01,0,79,1
2019-01-01,0,119,4
2019-01-01,0,120,3
2019-01-01,0,127,8
...,...,...,...
2019-12-31,2,3901,2
2019-12-31,2,3907,2
2019-12-31,2,3908,18


In [181]:
gender_count_df.loc[gender_count_df['gender'] == 0, 'gender'] = 'unknown'
gender_count_df.loc[gender_count_df['gender'] == 1, 'gender'] = 'male'
gender_count_df.loc[gender_count_df['gender'] == 2, 'gender'] = 'female'

In [182]:
gender_count_df

,gender,start station id,gCount
date,,,
2019-01-01,unknown,72,1
2019-01-01,unknown,79,1
2019-01-01,unknown,119,4
2019-01-01,unknown,120,3
2019-01-01,unknown,127,8
...,...,...,...
2019-12-31,female,3901,2
2019-12-31,female,3907,2
2019-12-31,female,3908,18


In [148]:
subscriber_count_df = gender_subscriber_df.groupby(['date','usertype', 'start station id']).count().rename(columns = {'gender': 'uCount'}).reset_index(level = [1, 2])

In [149]:
subscriber_count_df 

,usertype,start station id,uCount
date,,,
2019-01-01,Customer,72,11
2019-01-01,Customer,79,1
2019-01-01,Customer,82,2
2019-01-01,Customer,119,4
2019-01-01,Customer,120,4
...,...,...,...
2019-12-31,Subscriber,3907,2
2019-12-31,Subscriber,3908,39
2019-12-31,Subscriber,3909,2


In [150]:
subscriber_count_df.to_csv(path + usertype_count)

In [151]:
#gender_count_df = gender_count_df.reset_index()

In [183]:
gender_count_df.to_csv(path + gender_count)